In [68]:
import pandas as pd 

agrega_2016 = pd.read_csv('gs://trabalho-final-pdm/silver/agregados_2016.csv', storage_options={"token": "google_default"})
# Preparação: adicionar uma coluna de renda média (V005 + V009)
agrega_2016['RENDA'] = agrega_2016['V005'] + agrega_2016['V009']

agrega_2020 = pd.read_csv('gs://trabalho-final-pdm/silver/agregados_2020.csv', storage_options={"token": "google_default"})
# Preparação: adicionar uma coluna de renda média (V005 + V009)
agrega_2020['RENDA'] = agrega_2020['V005'] + agrega_2020['V009']

agrega_2024 = pd.read_csv('gs://trabalho-final-pdm/silver/agregados_2024.csv', storage_options={"token": "google_default"})
# Preparação: adicionar uma coluna de renda média (V005 + V009)
agrega_2024['RENDA'] = agrega_2024['V005'] + agrega_2024['V009']

In [35]:
agrega_2020.head()

,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,Nome_do_subdistrito,Situacao_setor_x,V005,V009,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,RENDA
0,2020,GOIANIA,135,260,2,1,VEREADOR,FERNANDO MARQUES FAUSTINO,40400,1,ESCOLA MUNICIPAL VIRGINIA GOMES PEREIRA,U.T.P. SANTO HILARIO,1.0,675.65,556.69,0.669983,0.510779,0.260365,1232.34
1,2020,GOIANIA,127,475,2,1,VEREADOR,JOAO BATISTA RODRIGUES,70234,1,COLEGIO META,U.T.P. JARDIM EUROPA,1.0,1538.41,986.92,0.456779,0.555050,0.207461,2525.33
2,2020,GOIANIA,146,311,2,1,VEREADOR,JOAO SANDES JUNIOR,11123,2,ESCOLA ESTADUAL JAIME CAMARA,U.T.P. BAIRRO FELIZ,1.0,2964.65,1716.74,0.389021,0.527446,0.193317,4681.39
3,2020,GOIANIA,1,470,2,1,VEREADOR,JOAO SANDES JUNIOR,11123,2,ESCOLAS DE ENGENHARIA CIVIL E ELETRICA - UFG,U.T.P. LESTE UNIVERSITARIO,1.0,2208.97,1637.20,0.406308,0.508349,0.257885,3846.17
4,2020,GOIANIA,1,470,2,1,VEREADOR,PAULO ROBERTO DE SOUSA,36361,1,ESCOLAS DE ENGENHARIA CIVIL E ELETRICA - UFG,U.T.P. LESTE UNIVERSITARIO,1.0,2208.97,1637.20,0.406308,0.508349,0.257885,3846.17


In [69]:
agrega = pd.concat([agrega_2016, agrega_2020, agrega_2024], ignore_index=True)

agrega = agrega[['ANO_ELEICAO', 'NM_VOTAVEL',  'NR_VOTAVEL', 'QT_VOTOS', 'Nome_do_subdistrito', 'Situacao_setor_x', 'Proporcao_negros_pardos_indigenas', 'Proporcao_feminino',
                'Proporcao_18_39', 'RENDA']]

In [70]:
agrega.shape

(969448, 10)

In [64]:
from google.cloud import bigquery
import pandas as pd

# Configuração
client = bigquery.Client()
table_ref = "pdm-class-2024.gold.tabela_com_anos_eleicao"

# Carregar o DataFrame para o BigQuery
job = client.load_table_from_dataframe(agrega, table_ref)
job.result()

print(f"Tabela {table_ref} exportada com sucesso!")

Tabela pdm-class-2024.gold.tabelas_sem_filtros exportada com sucesso!


In [65]:
# Salvar DataFrame diretamente no GCS
gcs_path = "gs://trabalho-final-pdm/gold/tabelas_sem_filtros.csv"

# Escrever para o Cloud Storage usando storage_options
agrega.to_csv(gcs_path, index=False, storage_options={"token": "google_default"})


In [ ]:
##############################################################

In [71]:
# Lista de partidos e seus respectivos números
partidos = {
    "15": "MDB",
    "12": "PDT",
    "13": "PT",
    "65": "PCdoB",
    "40": "PSB",
    "45": "PSDB",
    "36": "AGIR",
    "33": "MOBILIZA",
    "23": "CIDADANIA",
    "43": "PV",
    "70": "AVANTE",
    "11": "PP",
    "16": "PSTU",
    "21": "PCB",
    "28": "PRTB",
    "27": "DC",
    "29": "PCO",
    "20": "PODE",
    "10": "REPUBLICANOS",
    "50": "PSOL",
    "22": "PL",
    "55": "PSD",
    "77": "SOLIDARIEDADE",
    "30": "NOVO",
    "18": "REDE",
    "35": "PMB",
    "80": "UP",
    "44": "UNIÃO",
    "25": "PRD"
}

# Função para associar NR_VOTAVEL ao nome do partido ou "Coligações"
def identificar_num_partido(nr_votavel):
    str_votavel = str(nr_votavel).zfill(2)  # Garante pelo menos dois dígitos
    prefixo = str_votavel[:2]
    if prefixo in partidos:
        return int(prefixo), partidos[prefixo]
    return 0, "Coligações"

# Aplicar a transformação e criar a coluna 'PARTIDO'
agrega[['NUM_PARTIDO', 'PARTIDO']] = agrega['NR_VOTAVEL'].apply(lambda x: pd.Series(identificar_num_partido(x)))
                                                                

# Exibir a distribuição dos valores na nova coluna
partido_distribution = agrega['PARTIDO'].value_counts()
partido_distribution


PARTIDO
Coligações       132119
MDB               88860
PL                52102
DC                50438
PT                43515
PRD               40727
PSDB              40314
PSD               40313
AGIR              39737
PSB               38674
UNIÃO             37647
AVANTE            36976
PRTB              36271
REPUBLICANOS      35137
SOLIDARIEDADE     33740
PP                32533
PODE              30981
PV                25913
PDT               22885
MOBILIZA          22234
CIDADANIA         21642
PMB               16811
PSOL              14457
PCdoB             13933
NOVO              12776
REDE               6586
UP                 1657
PCB                 470
Name: count, dtype: int64

In [74]:
agrega['ANO_ELEICAO'].unique()

array([2016, 2020, 2024])

In [60]:
# Salvar DataFrame diretamente no GCS
gcs_path = "gs://trabalho-final-pdm/gold/tabelas_knn.csv"

# Escrever para o Cloud Storage usando storage_options
agrega.to_csv(gcs_path, index=False, storage_options={"token": "google_default"})

In [75]:
from google.cloud import bigquery
import pandas as pd

# Configuração
client = bigquery.Client()
table_ref = "pdm-class-2024.gold.tabelas_com_partidos"

# Carregar o DataFrame para o BigQuery
job = client.load_table_from_dataframe(agrega, table_ref)
job.result()

print(f"Tabela {table_ref} exportada com sucesso!")

Tabela pdm-class-2024.gold.tabelas_com_partidos exportada com sucesso!
